File for running a trip in SUMO and extracting relevant route features  
Author: Dennis Kibalama (kibalama.3@osu.edu)  
&copy; The Ohio State University, Center for Automotive Research 


In [2]:
# Import the required files and libraries
from termcolor import colored 
import os
import sys
import numpy as np
import scipy.io as scio
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib widget
import scipy.io
from xml.dom import minidom
# check SUMO_HOME 
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")
import traci # import TraCi
from env.TrafficSim import SumoEnv # importing the required python classes for interacting with SUMO environment

In [2]:
sumo_map = "reduced_columbus_region" # Name of the SUMO network file

# Define variables to save
host_distance = []
host_current_speed_limit = []
host_next_speed_imit = []
host_current_speed = []
sim_time = []
next_tl_distance = []
next_tl_state = []
route_long = []
route_lat = []
traffic_list = []
next_tl_index = []


# create_config_file(route_details=route_details, trip_number=trip)

env = SumoEnv(map_area=sumo_map, use_gui=False)
env.setup_sumo()
env.start_sumo()

# try:
step = 0
M = 20000

    # if not env.Done:
while step < M:
    
    env.simulation_step()
    if not env.done:
        host_current_speed_limit.append(env.current_speed_limit)
        host_current_speed.append(env.get_state()[0])
        host_distance.append(env.get_observations()[0])
        sim_time.append(step) 
        next_tl_distance.append(env.get_observations()[1])
        edge_list, edge_length_list = env.get_edge_attributes()
        
        # Only run this function once at the beginning of the simulation
        if step == 0:
            tl_route = env.get_traffic_light_location()

        # # append hyphen tnext_tl_state for easy extraction in MATLAB
        # next_tl_state.append("-")
        # next_tl_state.append(env.get_observations()[2])
        next_tl_state.extend(("-", env.get_observations()[2]))
        route_long.append(env.get_observations()[3])
        route_lat.append(env.get_observations()[4])
        traffic_list.append(env.get_observations()[5])
        # next_tl_index.append(env.get_observations()[6])

    if env.done:
        env.reset()
        break

    step += 1 # increase the simulation step

# convert to array
host_distance_np = np.array(host_distance)
host_current_speed_np = np.array(host_current_speed)
host_current_speed_limit_np = np.array(host_current_speed_limit)
sim_time_np = np.array(sim_time)
next_tl_distance_np = np.array(next_tl_distance)
next_tl_state_np = np.array(next_tl_state)
edge_length = np.array(edge_length_list)
route_long_np = np.array(route_long)
route_lat_np = np.array(route_lat)
tls_list = np.array(traffic_list[-1])
tl_route_np = np.around(tl_route, decimals=0)

# save the results to .mat file


save_directory = (f"{os.getcwd()}/results/")
save_dict = {"host_distance": np.array(host_distance_np), "sim_time": np.array(sim_time_np), "next_tl_distance": np.array(next_tl_distance_np), "host_current_speed_limit": np.array(host_current_speed_limit_np), "host_current_speed":np.array(host_current_speed_np), "next_tl_state": np.array(next_tl_state_np), "edge_length": edge_length, "tls_list": tls_list, "route_lat": route_lat_np, "route_long": route_long_np, "tl_route": tl_route_np}

# scipy.io.savemat(save_directory + "<vraribale_name>.mat", save_dict)

Vehicle host0 spawned!!
Traffic Light Locations [m]: [1614.42 2769.86 3240.16 3473.32 3632.89 9366.8  9692.36]
Simulation done!
